In [ ]:
# %pip install ollama
# %pip install gradio
# %pip install --upgrade gradio

In [ ]:
#region IMPORT

import gradio as gr
import random
import ollama

#endregion





#region CUSTOM
TEMPERATURE = 0.3

SYSTEM_PROMPT = """<|im_start|>system
    당신은 양세찬 게임 전문 참가자입니다. 반드시 다음 규칙을 따라야 합니다:

    1. 질문 생성 규칙
    - 단답형으로 답변 가능한 형태로 생성
    - 캐릭터 이름 직접 언급 금지
    - 주제와 관련된 내용

    2. 답변 규칙
    - 반드시 사실에 기반해 답변
    - 캐릭터 이름 직접 언급 금지
    - '예', '아니오' 또는 '단답형'으로 답변
    - 추가 설명 없이 5단어 이내로 응답

    3. 추측 규칙
    - 반드시 한 명의 '인물'로만 답변
    - 불확실하거나 정보가 부족하면 '모름'이라고 답변
    <|im_end|>
"""

# 자신 캐릭터 관련 질문 Prompt
QUESTION_PROMPT ="""<|im_start|>system
    양세찬 게임 질문 생성기로서 다음 규칙을 준수하세요:
    1. '예', '아니오' 또는 '단답형'으로 답변 가능한 단일 질문 생성
    2. 캐릭터 이름 직접 언급 금지
    3. 5단어 이내로 간결하게 표현<|im_end|>

    <|im_start|>user
    현재 주제: {topic}
    내 캐릭터를 유추할 수 있는 위 조건에 맞는 질문 하나를 생성하세요:<|im_end|>
"""

# 상대 질문(상대 캐릭터)에 대한 Prompt
ANSWER_PROMPT = """<|im_start|>system
    양세찬 게임 답변 생성기로서 다음 규칙을 준수하세요:
    1. '예', '아니오' 또는 '단답형'으로 응답
    2. 상대 캐릭터 이름 언급 금지
    3. 사실 기반 답변<|im_end|>

    <|im_start|>user
    현재 주제: {topic}
    상대 질문: {question}
    상대 캐릭터: {character}
    상대 질문에 대해 위 조건에 맞는 답변 하나를 생성하세요:<|im_end|>
"""

# 내 캐릭터 유추 Prompt
GUESS_PROMPT = """<|im_start|>system
    양세찬 게임 참가자로서 다음 규칙을 반드시 지켜서 답변하세요:
    1. 추가 설명 없이 반드시 한 명의 '인물' 또는 '가상인물'로 답변
    2. 정보가 부족한 경우 '모름'으로 답변
    3. 질문 형식 답변 금지
    4. '?' 포함 금지<|im_end|>
    
    <|im_start|>user
    현재 주제: {topic}
    현재까지의 정보만으로 내 캐릭터가 누구인지 유추추하고 답변하세요.
    
    아래는 예시입니다.
    - 유재석
    - 해리포터
    - 둘리
    - 김구
    - 박효신
    - 모름
    - 포레스트 검프
    - 토니 스타크
    - 캡틴 아메리카
    - 길라임
    - 이강
    - 채송화
    - 플라톤
    - 아리스토텔레스
    - 에디슨
    - 테슬라
    - 짱구
    - 곰돌이 푸
    - 미니언
    - 토토로<|im_end|>
"""

#endregion





#region CLASS

# User 클래스
class Player:
    def __init__(self, name):
        self.name = name
        self.character = ""

    def check_character(self, character):
        return self.character == character

# AI 클래스
class LLMPlayer(Player):
    MODEL_NAME = "EEVE-Korean-10.8B"
    
    def __init__(self, name):
        super().__init__(name)
        self.messages = [{"role": "system", "content": SYSTEM_PROMPT}]


    # 자신 캐릭터 관련 질문 (Question)
    def get_Q_res(self, content):
        self.messages.append({"role": "user", "content": content})

        q_res = ollama.chat(
            model=self.__class__.MODEL_NAME,
            messages=self.messages,
            options={
                "stop": ["<|im_start|>", "<|im_end|>", "<|eot_id|>"],
                "temperature": TEMPERATURE
            }
        )['message']['content'].strip()

        self.messages.append({"role": "assistant", "content": q_res})

        return q_res


    # 상대 질문에 대한 답 (Answer)
    def get_A_res(self, content):
        a_res = ollama.chat(
            model=self.__class__.MODEL_NAME,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": content}
            ],
            options={
                "stop": ["<|im_start|>", "<|im_end|>", "<|eot_id|>"],
                "temperature": TEMPERATURE
            }
        )['message']['content'].strip()

        return a_res


    # 내 캐릭터 유추 (Result)
    def get_R_res(self, content):
        self.messages.append({"role": "user", "content": content})

        r_res = ollama.chat(
            model=self.__class__.MODEL_NAME,
            messages=self.messages,
            options={
                "stop": ["<|im_start|>", "<|im_end|>", "<|eot_id|>"],
                "temperature": TEMPERATURE
            }
        )['message']['content'].strip()

        self.messages.append({"role": "assistant", "content": r_res})

        return r_res



    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

#endregion





#region VARIABLE

TOPICS = ["연예인", "영화 인물", "드라마 인물", "위인", "캐릭터"]

CHARACTERS = {
    "연예인": [
        "유재석", "아이유", "BTS 진", "블랙핑크 제니", "송강호", "김태리", "이영자", "박보검", "전지현", "이병헌",
        "공유", "한효주", "김수현", "김고은", "박명수", "이승기", "박서준", "정해인", "차은우", "수지"
    ],
    "영화 인물": [
        "해리 포터", "헤르미온느 그레인저", "론 위즐리", "인디아나 존스", "제임스 본드", "다스 베이더", "루크 스카이워커", "한 솔로", "조커", "배트맨",
        "포레스트 검프", "토니 스타크", "캡틴 아메리카", "엘사", "올라프", "도로시", "토토", "한니발 렉터", "토니 몬타나", "존 윅"
    ],
    "드라마 인물": [
        "김신", "지은탁", "이헌", "고애신", "도민준", "천송이", "성덕선", "최택", "박새로이", "조이서",
        "김주원", "길라임", "이강", "채송화", "이준혁", "차수현", "윤세리", "리정혁", "장만월", "구찬성"
    ],
    "위인": [
        "세종대왕", "이순신", "간디", "아인슈타인", "링컨", "나폴레옹", "클레오파트라", "칭기즈칸", "넬슨 만델라", "마리 퀴리",
        "플라톤", "아리스토텔레스", "에디슨", "테슬라", "다윈", "갈릴레이", "코페르니쿠스", "뉴턴", "파스퇴르", "멘델"
    ],
    "캐릭터": [
        "도라에몽", "피카츄", "뽀로로", "미키마우스", "슈퍼마리오", "소닉", "톰", "제리", "스폰지밥", "심슨",
        "짱구", "곰돌이 푸", "미니언", "토토로", "포뇨", "둘리", "고길동", "베지터", "카카로트", "루피"
    ]
}

GAME_STATE = {
    "players": {},
    "topic": "",
    "current_player_idx": 0,
    "round": 1,
    "game_started": False,
    "logs": [],
    "current_player": "",
    "winner": None
}

#endregion





#region FUNCTION

def combine_logs(logs):
    return "\n".join(logs)



def create_players(player_count):
    players = {}
    players["user"] = Player("user")
    for i in range(1, player_count):
        name = f"AI-{i}"
        players[name] = LLMPlayer(name)
    return players



def random_characters(topic, player_count):
    return random.sample(CHARACTERS[topic], k=player_count)



def start_game(topic, player_count):
    GAME_STATE["topic"] = topic
    GAME_STATE["players"] = create_players(player_count)
    GAME_STATE["round"] = 1
    GAME_STATE["current_player_idx"] = 0
    GAME_STATE["current_player"] = "user"
    GAME_STATE["game_started"] = True
    GAME_STATE["logs"] = []
    GAME_STATE["winner"] = None

    characters = random_characters(topic, player_count)
    for i, (name, player) in enumerate(GAME_STATE["players"].items()):
        player.character = characters[i]

    logs = []
    logs.append(f"게임 시작!!! 주제: '{topic}'")
    logs.append("각 플레이어에게 캐릭터를 할당했습니다.")

    for name, player in GAME_STATE["players"].items():
        logs.append(f"- {name} : {player.character}")

    logs.append(f"\n---------- {GAME_STATE['round']}라운드 ----------")
    logs.append(f"\n[{GAME_STATE['current_player']}의 차례]")
    logs.append("질문을 입력하세요.")
    GAME_STATE["logs"] = logs

    combined_log = combine_logs(logs)

    return (
        "게임이 시작되었습니다. 당신의 차례입니다!",
        combined_log,
        "",  # 입력창 초기화
        gr.Button("입력하기", interactive=True)  # 입력창 활성화
    )



def user_input_handler(user_input):
    if not GAME_STATE["game_started"] or GAME_STATE["winner"]:
        return "게임이 시작되지 않았거나 이미 종료되었습니다.", combine_logs(GAME_STATE["logs"]), "", gr.Button("입력하기", interactive=False)

    logs = GAME_STATE["logs"].copy()
    current_player = GAME_STATE["current_player"]

    # 내 차례일 때
    if (current_player == "user"):
        # 질문 단계
        if not logs or logs[-1].endswith("질문을 입력하세요."):
            logs.append(f"user 질문 >> {user_input}")

            for name, other_player in GAME_STATE["players"].items():
                if name == "user":
                    continue
                
                answer = other_player.get_A_res(
                    ANSWER_PROMPT.format(
                        character=GAME_STATE["players"]["user"].character,
                        question=user_input,
                        topic=GAME_STATE["topic"]
                    )
                )
                logs.append(f"{name} 답변 >> {answer}")

            logs.append("정답을 입력하세요.")
            GAME_STATE["logs"] = logs

            return "질문이 등록되었습니다. 정답을 입력하세요!", combine_logs(logs), "", gr.Button("입력하기", interactive=True)
        
        # 정답 단계
        elif logs[-1].endswith("정답을 입력하세요."):
            logs.append(f"user 정답 >> {user_input}")

            if user_input and user_input.strip() == GAME_STATE["players"]["user"].character:
                logs.append(f"🎉 'user'님이 정답을 맞췄습니다! 게임 종료")

                GAME_STATE["winner"] = "user"
                GAME_STATE["logs"] = logs
                
                return "정답입니다! 게임 종료", combine_logs(logs), "", gr.Button("입력하기", interactive=False)
            
            else:
                logs.append("정답이 아닙니다. 다음 플레이어로 넘어갑니다.")

                player_names = list(GAME_STATE["players"].keys())
                GAME_STATE["current_player_idx"] = (GAME_STATE["current_player_idx"] + 1) % len(player_names)
                GAME_STATE["current_player"] = player_names[GAME_STATE["current_player_idx"]]

                if GAME_STATE["current_player_idx"] == 0:
                    GAME_STATE["round"] += 1
                    logs.append(f"\n--- {GAME_STATE['round']}라운드 ---")
                
                logs.append(f"\n[{GAME_STATE['current_player']}의 차례]")
                GAME_STATE["logs"] = logs
                
                return ai_turn(), combine_logs(GAME_STATE["logs"]), "", gr.Button("입력하기", interactive=True)

    # AI 차례일 때 (user가 입력하면 무시)
    return "AI 차례입니다. 잠시만 기다려주세요.", combine_logs(logs), "", gr.Button("입력하기", interactive=False)



def ai_turn():
    logs = GAME_STATE["logs"]

    current_player = GAME_STATE["current_player"]
    if current_player == "user" or GAME_STATE["winner"]:
        if GAME_STATE["winner"]:
            return "게임이 종료되었습니다."
        else:
            logs.append("질문을 입력하세요.")
            return "당신의 차례입니다. 질문을 입력하세요."
    
    ai_player = GAME_STATE["players"][current_player]
    ai_question = ai_player.get_Q_res(
        QUESTION_PROMPT.format(
            topic=GAME_STATE["topic"]
        )
    )
    logs.append(f"{current_player} 질문 >> {ai_question}")

    if "user" in GAME_STATE["players"]:
        logs.append("질문에 대한 답변을 입력하세요.")
        GAME_STATE["logs"] = logs
        return f"'{current_player}'가 질문했습니다. 답변을 입력하세요."
    
    return "AI 턴 처리 완료"



def user_input_handler_ai_answer(user_input):
    logs = GAME_STATE["logs"].copy()
    current_player = GAME_STATE["current_player"]

    # AI 정답 추측
    if logs and logs[-1].endswith("질문에 대한 답변을 입력하세요."):
        logs.append(f"'user' 답변 >> {user_input}")

        ai_player = GAME_STATE["players"][current_player]
        ai_guess = ai_player.get_R_res(
            GUESS_PROMPT.format(
               topic=GAME_STATE["topic"]
            )
        )
        logs.append(f"{current_player} 정답 >> {ai_guess}")

        # 정답 판정
        if ai_guess and ai_guess.strip() == ai_player.character:
            logs.append(f"🎉 '{current_player}'가 정답을 맞췄습니다! 게임 종료")
            GAME_STATE["winner"] = current_player
            GAME_STATE["logs"] = logs
            return "AI가 정답을 맞췄습니다! 게임 종료", combine_logs(logs), "", gr.Button("입력하기", interactive=False)

        else:
            logs.append("정답이 아닙니다. 다음 플레이어로 넘어갑니다.")

            player_names = list(GAME_STATE["players"].keys())
            GAME_STATE["current_player_idx"] = (GAME_STATE["current_player_idx"] + 1) % len(player_names)
            GAME_STATE["current_player"] = player_names[GAME_STATE["current_player_idx"]]

            if GAME_STATE["current_player_idx"] == 0:
                GAME_STATE["round"] += 1
                logs.append(f"\n--- {GAME_STATE['round']}라운드 ---")

            logs.append(f"\n[{GAME_STATE['current_player']}의 차례]")
            GAME_STATE["logs"] = logs
            return ai_turn(), combine_logs(GAME_STATE["logs"]), "", gr.Button("입력하기", interactive=True)

    return user_input_handler(user_input)



def unified_input_handler(user_input):
    logs = GAME_STATE["logs"]
    if logs and logs[-1].endswith("질문에 대한 답변을 입력하세요."):
        return user_input_handler_ai_answer(user_input)
    
    return user_input_handler(user_input)



def start_game_with_custom(topic, player_count, temperature, system_prompt, question_prompt, answer_prompt, guess_prompt):
    # CUSTOM 변수 업데이트
    global TEMPERATURE, SYSTEM_PROMPT, QUESTION_PROMPT, ANSWER_PROMPT, GUESS_PROMPT
    TEMPERATURE = float(temperature)
    SYSTEM_PROMPT = system_prompt
    QUESTION_PROMPT = question_prompt
    ANSWER_PROMPT = answer_prompt
    GUESS_PROMPT = guess_prompt

    return start_game(topic, player_count)

#endregion





#region UI_GRADIO

with gr.Blocks(title="Prompt 학습 게임") as demo:
    gr.Markdown("# 양세찬 게임을 빙자한 Prompt 학습 게임")

    with gr.Row():
        topic_dropdown = gr.Dropdown(choices=TOPICS, label="주제 선택", value=TOPICS[0])
        player_count_slider = gr.Slider(minimum=2, maximum=4, step=1, value=3, label="플레이어 수 (사용자 포함)")
        start_button = gr.Button("게임 시작")

    status_text = gr.Textbox(label="게임 상태", interactive=False)

    with gr.Tabs():
        with gr.Tab("게임 로그"):
            log_textbox = gr.Textbox(label="전체 로그", interactive=False, lines=30)
            unified_input = gr.Textbox(label="입력", placeholder="질문, 답변 또는 정답을 입력하세요", interactive=True)
            submit_button = gr.Button("입력하기")

        with gr.Tab("게임 설정"):
            temperature_slider = gr.Slider(minimum=0.0, maximum=1.0, step=0.05, value=TEMPERATURE, label="Temperature")
            system_prompt_box = gr.Textbox(value=SYSTEM_PROMPT, label="System Prompt", lines=8)
            question_prompt_box = gr.Textbox(value=QUESTION_PROMPT, label="질문 프롬프트 - AI 모델이 할당받은 캐릭터 관련 질문", lines=6)
            answer_prompt_box = gr.Textbox(value=ANSWER_PROMPT, label="답변 프롬프트 - AI 모델이 상대 질문에 대한 답 관련", lines=6)
            guess_prompt_box = gr.Textbox(value=GUESS_PROMPT, label="추측 프롬프트 - AI 모델이 유추한 내 캐릭터", lines=8)

    start_button.click(
        fn=start_game_with_custom,
        inputs=[
            topic_dropdown, player_count_slider,
            temperature_slider, system_prompt_box, question_prompt_box, answer_prompt_box, guess_prompt_box
        ],
        outputs=[status_text, log_textbox, unified_input, submit_button]
    )

    submit_button.click(
        fn=unified_input_handler,
        inputs=[unified_input],
        outputs=[status_text, log_textbox, unified_input, submit_button]
    )

#endregion





#region MAIN

if __name__ == "__main__":
    demo.launch()

#endregion